In [1]:
import pandas as pd
import numpy as np
from os.path import join
import os
from rdkit import Chem
from rdkit.Chem import Crippen
from rdkit.Chem import Descriptors
from bioservices import *
from data_preprocessing import *
import warnings
warnings.filterwarnings('ignore')

CURRENT_DIR = os.getcwd()

c:\Users\marle\anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\marle\anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\marle\anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


## 1. Loading in Sabio data:

### (a) Loading Sabio data

In [2]:
organism = "Seed plants"

df_Sabio = pd.read_table(join("..", "..", "data", "Km_model_" + organism + ".tsv"))

df_Sabio["Km"] = df_Sabio["Km"].astype('float')
df_Sabio["PMID"] = df_Sabio["PMID"].astype('Int64')

df_Sabio["substrate_IDs"] = df_Sabio["substrate_IDs"].str.split('#')
df_Sabio["product_IDs"] = df_Sabio["product_IDs"].str.split('#')

print("Number of data points: %s" % len(df_Sabio))
print("Number of UniProt IDs: %s" % len(set(df_Sabio["Uniprot IDs"])))

df_Km = df_Sabio

Number of data points: 3635
Number of UniProt IDs: 835


#### Searching for identitcal pairs of UniProt IDs and Km values:

In [3]:
droplist = []

for ind in df_Km.index:
    UID, Km = df_Km["Uniprot IDs"][ind], df_Km["Km"][ind]
    help_df = df_Km.loc[df_Km["Uniprot IDs"] == UID].loc[df_Km["Km"] == Km]
    
    if len(help_df) > 1:
        droplist = droplist + list(help_df.index)[1:]
        

In [4]:
df_Km.drop(list(set(droplist)), inplace = True)
print("Dropping %s data points, because they are duplicated." % len(set(droplist)))
df_Km.reset_index(inplace = True, drop = True)
df_Km

Dropping 279 data points, because they are duplicated.


,ECs,Organism,Uniprot IDs,PMID,Type,Km,Temperature,pH,Substrates,Products,substrate_IDs,product_IDs,Main Substrate,Sequence
0,1,Petunia hybrida,Q15GI3,16782809,wildtype IGS1,0.000073,28.0,6.5,Coniferyl acetate;NADPH;,Isoeugenol;NADP+;Acetate;,[InChI=1S/C12H14O4/c1-9(13)16-7-3-4-10-5-6-11(...,[InChI=1S/C10H12O2/c1-3-4-8-5-6-9(11)10(7-8)12...,NADPH,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...
1,1,Petunia hybrida,Q15GI3,16782809,wildtype IGS1,0.001600,28.0,6.5,Coniferyl acetate;NADPH;,Isoeugenol;NADP+;Acetate;,[InChI=1S/C12H14O4/c1-9(13)16-7-3-4-10-5-6-11(...,[InChI=1S/C10H12O2/c1-3-4-8-5-6-9(11)10(7-8)12...,Coniferyl acetate,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...
2,1,Ocimum basilicum,Q15GI4,16782809,wildtype EGS1,0.005100,28.0,6.5,Coniferyl acetate;NADPH;,Eugenol;NADP+;Acetate;,[InChI=1S/C12H14O4/c1-9(13)16-7-3-4-10-5-6-11(...,[InChI=1S/C10H12O2/c1-3-4-8-5-6-9(11)10(7-8)12...,Coniferyl acetate,MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSSKTT...
3,1,Ocimum basilicum,Q15GI4,16782809,wildtype EGS1,0.000131,28.0,6.5,Coniferyl acetate;NADPH;,Eugenol;NADP+;Acetate;,[InChI=1S/C12H14O4/c1-9(13)16-7-3-4-10-5-6-11(...,[InChI=1S/C10H12O2/c1-3-4-8-5-6-9(11)10(7-8)12...,NADPH,MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSSKTT...
4,1.1.1,Vitis vinifera,Q1PSI9,16567629,wildtype his-tagged,0.002200,30.0,8.0,NAD+;L-Idonate;,NADH;5-Dehydro-D-gluconate;H+;,[InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-1...,[InChI=1S/C21H29N7O14P2/c22-17-12-19(25-7-24-1...,L-Idonate,MGKGGNSEDAVSGKEHGEENMAAWLLGIKTLKIQPYILPSLGPYDV...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3351,7.3.2.1,Arabidopsis thaliana,O82390,18353780,wildtype,0.000079,25.0,6.5,ATP;H2O;Phosphate;,ADP;Phosphate;,[InChI=1S/C10H16N5O13P3/c11-8-5-9(13-2-12-8)15...,[InChI=1S/C10H15N5O10P2/c11-8-5-9(13-2-12-8)15...,Phosphate,MNARALLCSSNIHSLYTSNRPPEKTSSSRSLRNLKPSPKSLRVWIY...
3352,7.3.2.1,Arabidopsis thaliana,O82390,18353780,wildtype,0.001170,25.0,6.5,ATP;H2O;Phosphate;,ADP;Phosphate;,[InChI=1S/C10H16N5O13P3/c11-8-5-9(13-2-12-8)15...,[InChI=1S/C10H15N5O10P2/c11-8-5-9(13-2-12-8)15...,Na+,MNARALLCSSNIHSLYTSNRPPEKTSSSRSLRNLKPSPKSLRVWIY...
3353,7.4.2.1,Arabidopsis thaliana,Q42400,12244056,wildtype AAP1,0.000774,30.0,4.5,ATP;L-Aspartate;H2O;,ADP;L-Aspartate;Phosphate;,[InChI=1S/C10H16N5O13P3/c11-8-5-9(13-2-12-8)15...,[InChI=1S/C10H15N5O10P2/c11-8-5-9(13-2-12-8)15...,L-Aspartate,MKSFNTEGHNHSTAESGDAYTVSDPTKNVDEDGREKRTGTWLTASA...
3354,7.4.2.1,Arabidopsis thaliana,O80592,12244056,wildtype AAP8,0.000444,30.0,4.5,ATP;L-Aspartate;H2O;,ADP;L-Aspartate;Phosphate;,[InChI=1S/C10H16N5O13P3/c11-8-5-9(13-2-12-8)15...,[InChI=1S/C10H15N5O10P2/c11-8-5-9(13-2-12-8)15...,L-Aspartate,MDAYNNPSAVESGDAAVKSVDDDGREKRTGTFWTASAHIITAVIGS...


In [5]:
def find_outliers_IQR(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers

find_outliers_IQR(df_Km["Km"])

print(df_Km['Km'].quantile(0.03),  df_Km['Km'].quantile(0.97))

1.1e-06 0.023104999999999973


In [6]:
df_Km = df_Km[(df_Km['Km'] > df_Km['Km'].quantile(0.03)) & (df_Km['Km'] < df_Km['Km'].quantile(0.97))]
df_Km.reset_index(inplace = True, drop = True)

In [7]:
todrop= []

for ind in df_Km.index:
    UID = df_Km["Uniprot IDs"][ind]
    if len(UID.split(';')) > 1:
        todrop.append(ind)
        print(df_Km["Uniprot IDs"][ind])
        print(todrop)
        
df_Km.drop(todrop, inplace=True)
df_Km.reset_index(inplace = True, drop = True)

P37225;P37221
[255]
P37225;P37221
[255, 256]
P37225;P37221
[255, 256, 257]
P37225;P37221
[255, 256, 257, 258]
P37225;P37221
[255, 256, 257, 258, 259]
P37225;P37221
[255, 256, 257, 258, 259, 260]
P37225;P37221
[255, 256, 257, 258, 259, 260, 261]
P37225;P37221
[255, 256, 257, 258, 259, 260, 261, 262]
P37225;P37221
[255, 256, 257, 258, 259, 260, 261, 262, 263]
P37225;P37221
[255, 256, 257, 258, 259, 260, 261, 262, 263, 264]
P37225;P37221
[255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265]
P37225;P37221
[255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266]
Q9ZRH5;Q8W000
[255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 302]
P17180;P00433;Q42517;P80679;P15232;P59121;P17179
[255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 302, 422]
P17180;P00433;Q42517;P80679;P15232;P59121;P17179
[255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 302, 422, 423]
P15233;P15232
[255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 302, 422, 423, 428]
P15233;P15

In [8]:
df_Km["substrate_IDs"] = df_Km["substrate_IDs"].apply(lambda x: (set(x)))
df_Km["product_IDs"] = df_Km["product_IDs"].apply(lambda x: (set(x)))

In [9]:
df_Km.to_pickle(join("..", "..", "data", "Km_data_merged.pkl"))

## 3. Assigning  IDs to every unique sequence and to every unique reaction in the dataset:

#### (a) Creating DataFrames for all sequences and for all reactions:

In [10]:
df_reactions = pd.DataFrame({"substrates": df_Km["substrate_IDs"],
                            "products" : df_Km["product_IDs"]})

df_reactions = df_reactions.loc[df_reactions["substrates"] != set([])]
df_reactions = df_reactions.loc[df_reactions["products"] != set([])]


droplist = []
for ind in df_reactions.index:
    sub_IDs, pro_IDs = df_reactions["substrates"][ind], df_reactions["products"][ind]
    help_df = df_reactions.loc[df_reactions["substrates"] == sub_IDs].loc[df_reactions["products"] == pro_IDs]
    if len(help_df):
        for ind in list(help_df.index)[1:]:
            droplist.append(ind)
            
df_reactions.drop(list(set(droplist)), inplace = True)
df_reactions.reset_index(inplace = True, drop =True)

df_reactions["Reaction ID"] = ["Reaction_" + str(ind) for ind in df_reactions.index]

In [11]:
df_reactions["min_Km_for_RID"] = np.nan
for ind in df_reactions.index:
    df_reactions["min_Km_for_RID"][ind] = min(df_Km.loc[df_Km["substrate_IDs"] == df_reactions["substrates"][ind]].loc[df_Km["product_IDs"] == df_reactions["products"][ind]]["Km"])

In [12]:
df_sequences = pd.DataFrame(data = {"Sequence" : df_Km["Sequence"].unique()})
df_sequences = df_sequences.loc[~pd.isnull(df_sequences["Sequence"])]
df_sequences.reset_index(inplace = True, drop = True)
df_sequences["Sequence ID"] = ["Sequence_" + str(ind) for ind in df_sequences.index]

df_sequences

,Sequence,Sequence ID
0,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,Sequence_0
1,MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSSKTT...,Sequence_1
2,MGKGGNSEDAVSGKEHGEENMAAWLLGIKTLKIQPYILPSLGPYDV...,Sequence_2
3,MANLRESSRDKSRWSLEGMTALVTGGSKGIGEAVVEELAMLGARVH...,Sequence_3
4,MAKEGGLGENSRWSLGGMTALVTGGSKGIGEAVVEELAMLGAKVHT...,Sequence_4
...,...,...
977,MSSLEDIKNETVDLEKIPIEEVFQQLKCSREGLTTQEGEDRIQIFG...,Sequence_977
978,MNARALLCSSNIHSLYTSNRPPEKTSSSRSLRNLKPSPKSLRVWIY...,Sequence_978
979,MKSFNTEGHNHSTAESGDAYTVSDPTKNVDEDGREKRTGTWLTASA...,Sequence_979
980,MDAYNNPSAVESGDAAVKSVDDDGREKRTGTFWTASAHIITAVIGS...,Sequence_980


In [13]:
df_sequences["min_Km_for_UID"] = np.nan
for ind in df_sequences.index:
    df_sequences["min_Km_for_UID"][ind] = min(df_Km.loc[df_Km["Sequence"] == df_sequences['Sequence'][ind]]["Km"])

#### (b) Calcuating the sum of the molecular weights of all substrates and of all products:

In [14]:
df_reactions["MW_frac"] = np.nan

for ind in df_reactions.index:
    substrates = list(df_reactions["substrates"][ind])
    products = list(df_reactions["products"][ind])
    
    mw_subs = mw_mets(metabolites = substrates)
    mw_pros = mw_mets(metabolites = products)
    
    if mw_subs == np.nan or mw_pros == np.nan:
        df_reactions["MW_frac"][ind] = np.inf
    if mw_pros != 0:
        df_reactions["MW_frac"][ind] = mw_subs/mw_pros
    else:
        df_reactions["MW_frac"][ind] = np.inf
        
df_reactions

[15:11:26] WARNING: not removing hydrogen atom without neighbors
[15:11:26] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not removing hydrogen atom without neighbors
[15:11:27] WARNING: not r

[15:11:29] WARNING: not removing hydrogen atom without neighbors
[15:11:29] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not removing hydrogen atom without neighbors
[15:11:30] WARNING: not r

,substrates,products,Reaction ID,min_Km_for_RID,MW_frac
0,{InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-1...,{InChI=1S/C21H28N7O17P3/c22-17-12-19(25-7-24-1...,Reaction_0,0.000073,1.001043
1,{InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-1...,{InChI=1S/C21H28N7O17P3/c22-17-12-19(25-7-24-1...,Reaction_1,0.000131,1.001043
2,{InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-1...,{InChI=1S/C6H10O7/c7-1-2(8)3(9)4(10)5(11)6(12)...,Reaction_2,0.002200,0.998829
3,{InChI=1S/C6H10O7/c7-1-2(8)3(9)4(10)5(11)6(12)...,{InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-1...,Reaction_3,0.012500,1.001173
4,"{InChI=1S/C7H12O/c1-6-3-2-4-7(8)5-6/h6H,2-5H2,...",{InChI=1S/C21H28N7O17P3/c22-17-12-19(25-7-24-1...,Reaction_4,0.000007,1.001175
...,...,...,...,...,...
848,"{InChI=1S/CH2O3/c2-1(3)4/h(H2,2,3,4)/p-1, InCh...","{InChI=1S/H3O4P/c1-5(2,3)4/h(H3,1,2,3,4), InCh...",Reaction_848,0.000011,0.996448
849,"{InChI=1S/CH2O3/c2-1(3)4/h(H2,2,3,4)/p-1, InCh...","{InChI=1S/H3O4P/c1-5(2,3)4/h(H3,1,2,3,4), InCh...",Reaction_849,0.000110,0.996412
850,"{InChI=1S/p+1, InChI=1S/C10H16N5O13P3/c11-8-5-...","{InChI=1S/H3O4P/c1-5(2,3)4/h(H3,1,2,3,4), InCh...",Reaction_850,0.000130,0.992338
851,"{InChI=1S/H3O4P/c1-5(2,3)4/h(H3,1,2,3,4), InCh...","{InChI=1S/H3O4P/c1-5(2,3)4/h(H3,1,2,3,4), InCh...",Reaction_851,0.000079,1.178908


#### (c) Mapping Sequence and Reaction IDs to Km_df:

In [15]:
import torch
import esm
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t33_650M_UR50D")

Using cache found in C:\Users\marle/.cache\torch\hub\facebookresearch_esm_main


In [16]:
#creating model input:
df_sequences["model_input"] = [seq[:1022] for seq in df_sequences["Sequence"]]
model_input = [(df_sequences["Sequence ID"][ind], df_sequences["model_input"][ind]) for ind in df_sequences.index]
seqs = [model_input[i][1] for i in range(len(model_input))]
#loading ESM-2 model:
print(".....2(a) Loading ESM-2 model.")
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
#convert input into batches:

#Calculate ESM-2 representations
print(".....2(b) Calculating enzyme representations.")
df_sequences["Enzyme rep"] = ""

for ind in df_sequences.index:
    batch_labels, batch_strs, batch_tokens = batch_converter([(df_sequences["Sequence ID"][ind], df_sequences["model_input"][ind])])
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33])
    df_sequences["Enzyme rep"][ind] = results["representations"][33][0, 1 : len(df_sequences["model_input"][ind]) + 1].mean(0).numpy()
    
df_sequences.head(5)

.....2(a) Loading ESM-2 model.
.....2(b) Calculating enzyme representations.


,Sequence,Sequence ID,min_Km_for_UID,model_input,Enzyme rep
0,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,Sequence_0,0.000073,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,"[-0.032205846, -0.031796537, -0.051493276, 0.0..."
1,MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSSKTT...,Sequence_1,0.000131,MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSSKTT...,"[-0.016749386, -0.048214775, -0.049711384, 0.0..."
2,MGKGGNSEDAVSGKEHGEENMAAWLLGIKTLKIQPYILPSLGPYDV...,Sequence_2,0.002200,MGKGGNSEDAVSGKEHGEENMAAWLLGIKTLKIQPYILPSLGPYDV...,"[-0.004863588, -0.06987531, -0.014163655, 0.06..."
3,MANLRESSRDKSRWSLEGMTALVTGGSKGIGEAVVEELAMLGARVH...,Sequence_3,0.000005,MANLRESSRDKSRWSLEGMTALVTGGSKGIGEAVVEELAMLGARVH...,"[-0.0007728094, -0.061243154, 0.041369718, 0.0..."
4,MAKEGGLGENSRWSLGGMTALVTGGSKGIGEAVVEELAMLGAKVHT...,Sequence_4,0.000030,MAKEGGLGENSRWSLGGMTALVTGGSKGIGEAVVEELAMLGAKVHT...,"[-0.011831572, -0.06318858, 0.038726494, 0.021..."


In [19]:
def get_metabolite_type(met):
    if is_KEGG_ID(met):
        return("KEGG")
    elif is_InChI(met):
        return("InChI")
    else:
        return("invalid")

def get_reaction_site_smarts(metabolites):
    reaction_site = ""
    for met in metabolites:
        met_type = get_metabolite_type(met)
        if met_type == "KEGG":
            try:
                Smarts = Chem.MolToSmarts(Chem.MolFromMolFile(join(CURRENT_DIR, "data", "mol-files",  met + ".mol")))
            except OSError:
                return(np.nan)
        elif met_type == "InChI":
            Smarts = Chem.MolToSmarts(Chem.inchi.MolFromInchi(met))
        else:
            Smarts = "invalid"
        reaction_site = reaction_site + "." + Smarts
    return(reaction_site[1:])


def is_KEGG_ID(met):
    #a valid KEGG ID starts with a "C" or "D" followed by a 5 digit number:
    if len(met) == 6 and met[0] in ["C", "D"]:
        try:
            int(met[1:])
            return(True)
        except: 
            pass
    return(False)

def is_InChI(met):
    m = Chem.inchi.MolFromInchi(met,sanitize=False)
    if m is None:
      return(False)
    else:
      try:
        Chem.SanitizeMol(m)
      except:
        print('.......Metabolite string "%s" is in InChI format but has invalid chemistry' % met)
        return(False)
    return(True)

def convert_fp_to_array(difference_fp_dict):
    fp = np.zeros(2048)
    for key in difference_fp_dict.keys():
        fp[key] = difference_fp_dict[key]
    return(fp)

In [20]:
from rdkit.Chem import AllChem

df_reactions["difference_fp"], df_reactions["structural_fp"],  = "", ""
#each metabolite should be either a KEGG ID, InChI string, or a SMILES:
for ind in df_reactions.index:
    left_site = get_reaction_site_smarts(df_reactions["substrates"][ind])
    right_site = get_reaction_site_smarts(df_reactions["products"][ind])
    if not pd.isnull(left_site) and not pd.isnull(right_site):
        rxn_forward = AllChem.ReactionFromSmarts(left_site + ">>" + right_site)
        difference_fp = Chem.rdChemReactions.CreateDifferenceFingerprintForReaction(rxn_forward)
        difference_fp = convert_fp_to_array(difference_fp.GetNonzeroElements())
        df_reactions["difference_fp"][ind] = difference_fp
        df_reactions["structural_fp"][ind] = Chem.rdChemReactions.CreateStructuralFingerprintForReaction(rxn_forward).ToBitString()

df_reactions.head(5)

[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not removing hydrogen atom without neighbors
[17:42:00] WARNING: not r

[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not removing hydrogen atom without neighbors
[17:42:04] WARNING: not r

,substrates,products,Reaction ID,min_Km_for_RID,MW_frac,difference_fp,structural_fp
0,{InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-1...,{InChI=1S/C21H28N7O17P3/c22-17-12-19(25-7-24-1...,Reaction_0,0.000073,1.001043,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001111111100...
1,{InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-1...,{InChI=1S/C21H28N7O17P3/c22-17-12-19(25-7-24-1...,Reaction_1,0.000131,1.001043,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001111111100...
2,{InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-1...,{InChI=1S/C6H10O7/c7-1-2(8)3(9)4(10)5(11)6(12)...,Reaction_2,0.002200,0.998829,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001101111100...
3,{InChI=1S/C6H10O7/c7-1-2(8)3(9)4(10)5(11)6(12)...,{InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-1...,Reaction_3,0.012500,1.001173,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001111111100...
4,"{InChI=1S/C7H12O/c1-6-3-2-4-7(8)5-6/h6H,2-5H2,...",{InChI=1S/C21H28N7O17P3/c22-17-12-19(25-7-24-1...,Reaction_4,0.000007,1.001175,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001111111100...


In [21]:
df_Km = df_Km.merge(df_sequences, on = "Sequence", how = "left")

In [22]:
df_reactions.rename(columns = {"substrates" : "substrate_IDs",
                              "products" : "product_IDs"}, inplace = True)

df_Km["Reaction ID"] = np.nan
df_Km["MW_frac"] = np.nan
df_Km["min_Km_for_RID"] = np.nan
df_Km["difference_fp"] = ""
df_Km["structural_fp"] = ""

for ind in df_Km.index:
    sub_set, pro_set = df_Km["substrate_IDs"][ind], df_Km["product_IDs"][ind]
    help_df = df_reactions.loc[df_reactions["substrate_IDs"] == sub_set].loc[df_reactions["product_IDs"] == pro_set]
    if len(help_df) == 1:
        df_Km["Reaction ID"][ind] = list(help_df["Reaction ID"])[0]
        df_Km["min_Km_for_RID"][ind] = list(help_df["min_Km_for_RID"])[0]
        df_Km["MW_frac"][ind] = list(help_df["MW_frac"])[0]
        df_Km["difference_fp"][ind] = list(help_df["difference_fp"])[0]
        df_Km["structural_fp"][ind] = list(help_df["structural_fp"])[0]
df_Km.head(2)

,ECs,Organism,Uniprot IDs,PMID,Type,Km,Temperature,pH,Substrates,Products,...,Sequence,Sequence ID,min_Km_for_UID,model_input,Enzyme rep,Reaction ID,MW_frac,min_Km_for_RID,difference_fp,structural_fp
0,1,Petunia hybrida,Q15GI3,16782809,wildtype IGS1,0.000073,28.0,6.5,Coniferyl acetate;NADPH;,Isoeugenol;NADP+;Acetate;,...,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,Sequence_0,0.000073,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,"[-0.032205846, -0.031796537, -0.051493276, 0.0...",Reaction_0,1.001043,0.000073,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001111111100...
1,1,Petunia hybrida,Q15GI3,16782809,wildtype IGS1,0.001600,28.0,6.5,Coniferyl acetate;NADPH;,Isoeugenol;NADP+;Acetate;,...,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,Sequence_0,0.000073,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,"[-0.032205846, -0.031796537, -0.051493276, 0.0...",Reaction_0,1.001043,0.000073,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001111111100...


In [ ]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys

df_Km["MACCS FP"] = ""

for ind in df_Km.index:
    substrate = df_Km["Main substrate"][ind]
    id = df_Km['Substrate_IDs'][ind][df_Km["Substrates"][ind].split(';').index(substrate)]
    if id[0] == "C":
        try:
            mol = Chem.MolFromMolFile("../../data/mol-files/" + id + '.mol')
        except OSError:
            None
    else:
        try:
            mol = Chem.inchi.MolFromInchi(id,sanitize=False)
        except OSError:
            None
    if mol is not None:
        maccs_fp = MACCSkeys.GenMACCSKeys(mol).ToBitString()
        df_Km["MACCS FP"][ind] = maccs_fp

#### (d) Creating a new DataFrame with one entry for every unique sequence-reaction pair:

##### (d)(i) Creating the DataFrame:

In [23]:
# df_Km_new = pd.DataFrame(data = {"Reaction ID" : df_Km["Reaction ID"],
#                                   "Sequence ID" : df_Km["Sequence ID"],
#                                   "Temperature" : df_Km["Temperature"],
#                                     "pH" : df_Km["pH"]})
# df_Km_new = df_Km_new.loc[~pd.isnull(df_Km_new["Reaction ID"])].loc[~pd.isnull(df_Km_new["Sequence ID"])]
# df_Km_new.drop_duplicates(inplace = True)
# df_Km_new.reset_index(inplace = True, drop = True)


# df_Km_new["Km_values"], df_Km_new["Uniprot IDs"], df_Km_new["ECs"], df_Km_new["Organisms"] = "", "", "", ""

# for ind in df_Km_new.index:
#     RID, SID = df_Km_new["Reaction ID"][ind], df_Km_new["Sequence ID"][ind]
#     help_df = df_Km.loc[df_Km["Reaction ID"] == RID].loc[df_Km["Sequence ID"] == SID]
#     df_Km_new["ECs"][ind] = list(help_df["ECs"])
#     df_Km_new["Km_values"][ind] = list(help_df["Km"])
#     df_Km_new["Uniprot IDs"][ind] = list(help_df["Uniprot IDs"])
#     df_Km_new["Organism"][ind] = list(help_df["Organism"])
# df_Km_new

##### (d)(ii): Adding sequence, substrates, and products to all data points

In [24]:
# df_Km_new["Sequence"], df_Km_new["substrates"], df_Km_new["products"], df_Km_new["MW_frac"] = "", "", "", ""

# for ind in df_Km_new.index:
#     RID, SID = df_Km_new["Reaction ID"][ind], df_Km_new["Sequence ID"][ind]
#     print(RID,SID)
#     help_df = df_reactions.loc[df_reactions["Reaction ID"] == RID]
#     print(help_df)
#     df_Km_new["substrates"][ind], df_Km_new["products"][ind] = list(help_df["substrate_IDs"])[0], list(help_df["product_IDs"])[0]
#     df_Km_new["MW_frac"][ind] = list(help_df["MW_frac"])[0]
    
#     help_df = df_sequences.loc[df_sequences["Sequence ID"] == SID]
#     df_Km_new["Sequence"][ind] = list(help_df["Sequence"])[0]
    

##### (d)(iii) Calculating the minimal Km value for every sequence and for every reaction:

In [25]:
# df_all_Km = pd.read_pickle(join("..", "..", "data", "Km_data_merged.pkl"))
# df_all_Km.head()

# df_Km_new["min_Km_for_UID"]  = ""
# df_Km_new["min_Km_for_RID"]  = ""

# for ind in df_Km_new.index:
#     min_Km = np.inf
#     UIDs = list(set(df_Km_new["Uniprot IDs"][ind]))
#     for UID in UIDs:
#         all_Km = list(df_all_Km["Km"].loc[df_all_Km["Uniprot IDs"] == UID])
#         all_Km = [float(Km) for Km in all_Km]
#         min_Km = min(min_Km, min(all_Km))
#     df_Km_new["min_Km_for_UID"][ind] = min_Km


# for ind in df_Km_new.index:
#     RID = df_Km_new["Reaction ID"][ind]
    
#     help_df = df_Km_new.loc[df_Km_new["Reaction ID"] == RID]
#     all_Km = []
#     for ind2 in help_df.index:
#         all_Km = all_Km + list(help_df["Km_values"][ind2])
#     all_Km = [float(Km) for Km in all_Km]
#     min_Km = min(all_Km)
#     df_Km_new["min_Km_for_RID"][ind] = min_Km
# df_Km_new.head(20)

##### (d)(iv) Calculating the minimal Km value for every EC number in the dataset:

In [26]:
# df_Km = df_Km_new.copy()

df_Km.head()

,ECs,Organism,Uniprot IDs,PMID,Type,Km,Temperature,pH,Substrates,Products,...,Sequence,Sequence ID,min_Km_for_UID,model_input,Enzyme rep,Reaction ID,MW_frac,min_Km_for_RID,difference_fp,structural_fp
0,1,Petunia hybrida,Q15GI3,16782809,wildtype IGS1,0.000073,28.0,6.5,Coniferyl acetate;NADPH;,Isoeugenol;NADP+;Acetate;,...,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,Sequence_0,0.000073,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,"[-0.032205846, -0.031796537, -0.051493276, 0.0...",Reaction_0,1.001043,0.000073,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001111111100...
1,1,Petunia hybrida,Q15GI3,16782809,wildtype IGS1,0.001600,28.0,6.5,Coniferyl acetate;NADPH;,Isoeugenol;NADP+;Acetate;,...,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,Sequence_0,0.000073,MTTGKGKILILGATGYLGKYMVKASISLGHPTYAYVMPLKKNSDDS...,"[-0.032205846, -0.031796537, -0.051493276, 0.0...",Reaction_0,1.001043,0.000073,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001111111100...
2,1,Ocimum basilicum,Q15GI4,16782809,wildtype EGS1,0.005100,28.0,6.5,Coniferyl acetate;NADPH;,Eugenol;NADP+;Acetate;,...,MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSSKTT...,Sequence_1,0.000131,MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSSKTT...,"[-0.016749386, -0.048214775, -0.049711384, 0.0...",Reaction_1,1.001043,0.000131,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001111111100...
3,1,Ocimum basilicum,Q15GI4,16782809,wildtype EGS1,0.000131,28.0,6.5,Coniferyl acetate;NADPH;,Eugenol;NADP+;Acetate;,...,MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSSKTT...,Sequence_1,0.000131,MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSSKTT...,"[-0.016749386, -0.048214775, -0.049711384, 0.0...",Reaction_1,1.001043,0.000131,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001111111100...
4,1.1.1,Vitis vinifera,Q1PSI9,16567629,wildtype his-tagged,0.002200,30.0,8.0,NAD+;L-Idonate;,NADH;5-Dehydro-D-gluconate;H+;,...,MGKGGNSEDAVSGKEHGEENMAAWLLGIKTLKIQPYILPSLGPYDV...,Sequence_2,0.002200,MGKGGNSEDAVSGKEHGEENMAAWLLGIKTLKIQPYILPSLGPYDV...,"[-0.004863588, -0.06987531, -0.014163655, 0.06...",Reaction_2,0.998829,0.002200,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1100111100000001001000110110010001001101111100...


In [27]:
df_EC_Km = pd.read_csv(join("..", "..", "data", "min_EC_" + organism + ".tsv"), sep = "\t", header=0)
# df_EC_Km = df_EC_Km.rename(columns={0: "EC", 1: "min_Km"})

for ind in df_EC_Km.index:
    try:
        Km_min = df_EC_Km[df_EC_Km["EC"] == df_Km["ECs"]]["min_Km"]
        df_EC_Km["min_Km"][ind] = Km_min
        print(ind, Km_min)
    except:
        pass
        
df_EC_Km.describe()

,min_Km
count,541.000000
mean,0.000498
std,0.003011
min,0.000000
25%,0.000003
50%,0.000019
75%,0.000100
max,0.050000


Mapping min EC Km value to all data points:

In [28]:
df_EC_Km = pd.read_csv(join("..", "..", "data", "min_EC_" + organism + ".tsv"), sep = "\t", header=0)

df_EC_Km.head(5)
df_Km["min_Km_for_EC"] = np.nan

for ind in df_Km.index:
    EC = df_Km["ECs"][ind]
    min_Km = 0
    try:
        print(EC)
        min_Km = df_EC_Km.loc[df_EC_Km["EC"] == EC, "min_Km"].iloc[0]
        print(min_Km)
    except:
        pass
    if min_Km != 0:
        df_Km["min_Km_for_EC"][ind] = min_Km
df_Km.to_pickle(join("..", "..", "data", "merged_and_grouped_Km_dataset.pkl"))     

1
7.3e-05
1
7.3e-05
1
7.3e-05
1
7.3e-05
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07
1.1.1
3e-07


1.731e-05
2.1.1.68
1.731e-05
2.1.1.68
1.731e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.7
3.654e-05
2.1.1.89
3.5e-05
2.1.1.n7
2.3e-05
2.1.1.n7
2.3e-05
2.1.1.n7
2.3e-05
2.1.1.n7
2.3e-05
2.1.1.n7
2.3e-05
2.1.1.n7
2.3e-05
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.1.2.1
5.5e-07
2.10.1.1
0.000133
2.10.1.1
0.000133
2.2.1.1
6.7e-05
2.2.1.1
6.7e-05
2.2.1.6
0.0062
2.2.1.6
0.0062
2.2.1.6
0.0062
2.2.1.6
0.0062
2.2.1.6
0.0062
2.3.1
5e-06
2.3.1
5e-06
2.3.1
5e-0

2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.47
3.5e-06
2.5.1.48
0.00018
2.5.1.48
0.00018
2.5.1.48
0.00018
2.5.1.48
0.00018
2.5.1.48
0.00018
2.5.1.48
0.00018
2.5.1.48
0.00018
2.5.1.48
0.00018
2.5.1.48
0.00018
2.5.1.48
0.00018
2.5.1.93
5.1e-06
2.5.1.93
5.1e-06
2.5.1.93
5.1e-06
2.5.1.93
5.1e-06
2.5.1.93
5.1e-06
2.5.1.93
5.1e-06
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2e-05
2.6.1.1
2

3.1.3.25
6.1e-05
3.1.3.36
0.000101
3.1.3.41
0.015
3.1.3.41
0.015
3.1.3.46
2.4e-08
3.1.3.46
2.4e-08
3.1.3.56
0.000233
3.1.3.56
0.000233
3.1.3.56
0.000233
3.1.3.86
0.000211
3.1.3.93
2.2e-05
3.1.3.93
2.2e-05
3.1.3.93
2.2e-05
3.1.3.93
2.2e-05
3.1.4.37
0.0092
3.1.7.11
2.1e-05
3.1.7.11
2.1e-05
3.1.7.11
2.1e-05
3.1.7.11
2.1e-05
3.2.1.147
2.3e-05
3.2.1.147
2.3e-05
3.2.1.147
2.3e-05
3.2.1.161
9e-05
3.2.1.161
9e-05
3.2.1.161
9e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.182
9.8e-05
3.2.1.20
0.0037
3.2.1.20
0.0037
3.2.1.20
0.0037
3.2.1.20
0.0037
3.2.1.20
0.0037
3.2.1.206
0.00248
3.2.1.21
3.1e-05
3.2.1.21
3.1e-05
3.2.1.21
3.1e-05
3.2.1.21
3.1e-05
3.2.1.21
3.1e-05
3.2.1.21
3.1e-

1e-05
4.3.1.24
1e-05
4.3.1.24
1e-05
4.3.1.25
1.9e-05
4.3.1.25
1.9e-05
4.3.1.25
1.9e-05
4.3.1.25
1.9e-05
4.3.2.7
0.0017
4.3.2.7
0.0017
4.3.3.7
0.0004
4.3.3.7
0.0004
4.4.1.1
0.0004
4.4.1.14
8.3e-06
4.4.1.14
8.3e-06
4.4.1.14
8.3e-06
4.4.1.14
8.3e-06
4.4.1.14
8.3e-06
4.4.1.14
8.3e-06
4.4.1.14
8.3e-06
4.4.1.14
8.3e-06
4.4.1.14
8.3e-06
4.4.1.14
8.3e-06
4.4.1.14
8.3e-06
4.4.1.16
0.0002
4.4.1.16
0.0002
4.4.1.3
0.00022
4.4.1.3
0.00022
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.4.1.9
3.1e-05
4.6.1.12
0.000483
5.1.3.1
0.00025
5.1.3.18
4.5e-06
5.1.3.18
4.5e-06
5.1.3.18
4.5e-06
5.1.3.18
4.5e-06
5.1.3.18
4.5e-06
5.1.3.2
4e-05
5.1.3.2
4e-05
5.1.3.2
4e-05
5.1.3.2
4e-05
5.1.3.2
4e-05
5.1.3.2
4e-05
5.1.3.2
4e-05
5.1.3.2
4e-05
5.1.3.2
4e-05
5.1.3

In [29]:
df_sequences.to_pickle(join("..", "..", "data", "all_sequences_with_IDs.pkl"))
df_reactions.to_pickle(join("..", "..", "data", "all_reactions_with_IDs.pkl"))

## 5.Removing outliers and non-natural reactions:

In [15]:
df_Km = pd.read_pickle(join("..", "..", "data", "merged_and_grouped_Km_dataset.pkl"))

C:\Users\marle\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Glucose D-Glucose
81 1
['InChI=1S/p+1', 'InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2/t2-,3-,4+,5-,6?/m1/s1', 'InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(44-46(33,34)35)14(30)11(43-21)6-41-48(38,39)45-47(36,37)40-5-10-13(29)15(31)20(42-10)27-3-1-2-9(4-27)18(23)32/h1,3-4,7-8,10-11,13-16,20-21,29-31H,2,5-6H2,(H2,23,32)(H,36,37)(H,38,39)(H2,22,24,25)(H2,33,34,35)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1']
D-Glucose 6-phosphate Glucose 6-phosphate
336 1
['InChI=1S/C6H13O9P/c7-3-2(1-14-16(11,12)13)15-6(10)5(9)4(3)8/h2-10H,1H2,(H2,11,12,13)/t2-,3-,4+,5-,6?/m1/s1', 'InChI=1S/C21H28N7O17P3/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(44-46(33,34)35)14(30)11(43-21)6-41-48(38,39)45-47(36,37)40-5-10-13(29)15(31)20(42-10)27-3-1-2-9(4-27)18(23)32/h1-4,7-8,10-11,13-16,20-21,29-31H,5-6H2,(H7-,22,23,24,25,32,33,34,35,36,37,38,39)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1']
L-Serine Serine
996 1
['InChI=1S/C3H7NO3/c4-2(1-5)3(6)7/h2,5H,1,4H2,(H,6,7)/t2-/m0/s1', 'InChI=1S/C19H23N

#### (a) Calculating the geometric mean and log10-transforming it for all enzyme-reaction pairs:
To ignore $Km$ values that were obtained under non-optimal conditions, we exclude values higher than 10000\% than the minimal $Km$ value for the same enzyme-reaction combination.

In [16]:
df_Km["frac_of_min_UID"] = np.nan
df_Km["frac_of_min_RID"] = np.nan
df_Km["frac_of_min_EC"] = np.nan

for ind in df_Km.index:
    df_Km["frac_of_min_UID"][ind] =  df_Km["min_Km_for_UID"][ind]/df_Km["Km"][ind]
    df_Km["frac_of_min_RID"][ind] =  df_Km["min_Km_for_RID"][ind]/df_Km["Km"][ind]
    df_Km["frac_of_min_EC"][ind] = df_Km["min_Km_for_EC"][ind]/df_Km["Km"][ind]

len(df_Km)

C:\Users\marle\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\marle\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\marle\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


2891

#### (b) We are only interested in Km values that were measured for the natural reaction of an enzyme:
To achieve this we exclude Km values for an enzyme if another measurement exists for the same enzyme but for different reaction with a Km value that is more than ten times lower. Furthermore, to exlcude data points measured under non-optimal conditions and for non-natural reactions, we exclude Km values if we could find a Km value for the same reaction or same EC number that is more than 100 times lower.

In [17]:
n = len(df_Km)

df_Km = df_Km.loc[df_Km["frac_of_min_UID"] >= 0.01]
df_Km = df_Km.loc[df_Km["frac_of_min_RID"] >= 0.01]

df_Km["frac_of_min_EC"].loc[pd.isnull(df_Km["frac_of_min_EC"])] = 1
df_Km = df_Km.loc[df_Km["frac_of_min_EC"] <= 10]
df_Km = df_Km.loc[df_Km["frac_of_min_EC"] >= 0.01]

In [18]:
print("We remove %s data points, because we suspect that these Km values were not measure for the natural reaction " \
    "of an enzyme or under non-optimal conditions." % (n-len(df_Km)))

We remove 621 data points, because we suspect that these Km values were not measure for the natural reaction of an enzyme or under non-optimal conditions.


#### (c) Removing data points with reaction queations with uneven fraction of molecular weights

In [19]:
n = len(df_Km)

df_Km = df_Km.loc[df_Km["MW_frac"] < 3]
df_Km = df_Km.loc[df_Km["MW_frac"] > 1/3]

print("We remove %s data points because the sum of molecular weights of substrates does not match the sum of molecular" \
      "weights of the products." % (n-len(df_Km)))

We remove 123 data points because the sum of molecular weights of substrates does not match the sum of molecularweights of the products.


In [21]:
print("Size of final Km dataset: %s" % len(df_Km))
df_Km.to_pickle(join("..", "..", "data", "final_Km_dataset_" + organism + ".pkl"))

Size of final Km dataset: 2147


In [20]:
df_Km = pd.read_pickle(join("..", "..", "data", "final_Km_dataset_" + organism + ".pkl"))

In [22]:
df_Km.to_csv('../data_' + organism + '.csv', index=False)

In [21]:
df_Km["Type"][df_Km['Type'].str.contains("wildtype")] = "wildtype"
df_Km["Type"][df_Km['Type'].str.contains("mutant")] = "mutant"

In [24]:
df_Arabidopsis = df_Km[df_Km["Organism"] == 'Arabidopsis thaliana']
df_validation = df_Km[df_Km["Organism"] != 'Arabidopsis thaliana']

In [22]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

organisms = {}

def is_brassicaceae(org):
    try:
        tax_id = ncbi.get_name_translator([org])[org][0]
        lineage = ncbi.get_lineage(tax_id)
        if 3700 not in lineage:
            return(False)
        else:
            return(True)
    except KeyError:
        return(False)
    
for org in df_Km["Organism"].tolist():
    if org not in organisms.keys():
        organisms[org] = is_brassicaceae(org)

In [23]:
# train_df = df_Km[df_Km["Organism"].isin([key for key, value in organisms.items() if value is False])]
# test_df = df_Km[df_Km["Organism"].isin([key for key, value in organisms.items() if value is True])]
# train_df.reset_index(inplace = True, drop = True)
# test_df.reset_index(inplace = True, drop = True)
df_Km = df_Km[df_Km["Organism"].isin([key for key, value in organisms.items() if value is True])]
df_Km = df_Km[df_Km["Type"] == 'wildtype']
df_Km.reset_index(inplace = True, drop = True)

In [11]:
import random
random.seed(10)
ind = random.sample(range(len(test_df)), len(test_df)//2)
train_df = pd.concat([train_df,test_df.loc[test_df.index[ind]]])
test_df = test_df[~test_df.index.isin(ind)]
train_df.reset_index(inplace = True, drop = True)
test_df.reset_index(inplace = True, drop = True)

In [30]:
df_new = pd.DataFrame(data = {"Reaction ID" : df_Arabidopsis["Reaction ID"],
                                  "Sequence ID" : df_Arabidopsis["Sequence ID"],
                                  "Temperature" : df_Arabidopsis["Temperature"],
                                    "pH" : df_Arabidopsis["pH"],
                                 "Type": df_Arabidopsis["Type"],
                             "Main substrate" : df_Arabidopsis["MACCS FP"]})

df_new.drop_duplicates(inplace = True)
df_new.reset_index(inplace = True, drop = True)

df_new["Km_values"], df_new["Uniprot IDs"], df_new["ECs"], df_new["Substrates"], df_new["Products"], df_new["ESM2"], df_new["Sequence"], df_new["difference_fp"], df_new["structural_fp"] = "", "", "", "", "", "", "", "", ""

for ind in df_new.index:
    RID, SID, Temp, pH, Type, MSubstrate = df_new["Reaction ID"][ind], df_new["Sequence ID"][ind], df_new["Temperature"][ind], df_new["pH"][ind], df_new["Type"][ind], df_new["Main substrate"][ind]
    help_df = df_Arabidopsis.loc[df_Arabidopsis["Reaction ID"] 
                                 == RID].loc[df_Arabidopsis["Sequence ID"] 
                                             == SID].loc[df_Arabidopsis["Temperature"] 
                                                         == Temp].loc[df_Arabidopsis["pH"] 
                                                                      == pH].loc[df_Arabidopsis["Type"] 
                                                                                 == Type].loc[df_Arabidopsis["MACCS FP"] 
                                                                                              == MSubstrate]
    print(help_df)
    df_new["ECs"][ind] = list(help_df["ECs"])
    df_new["Km_values"][ind] = list(help_df["Km"])
    df_new["Uniprot IDs"][ind] = list(help_df["Uniprot IDs"])
    df_new["Sequence"][ind] = help_df["Sequence"].values[0]
    df_new["ESM2"][ind] = help_df["Enzyme rep"].values[0]
    df_new["difference_fp"][ind], df_new["structural_fp"][ind] = help_df["difference_fp"].values[0], help_df["structural_fp"].values[0]
    df_new["Substrates"][ind], df_new["Products"][ind] = help_df["Substrates"].values[0], help_df["Products"].values[0]

      ECs              Organism Uniprot IDs      PMID      Type        Km  \
12  1.1.1  Arabidopsis thaliana      Q9ZW03  24583623  wildtype  0.000007   

   Temperature   pH                       Substrates  \
12        30.0  5.0  NADPH;3-Methylcyclohexanone;H+;   

                       Products  ... Reaction ID   MW_frac min_Km_for_RID  \
12  NADP+;3-Methylcyclohexanol;  ...  Reaction_4  1.001175       0.000007   

                                        difference_fp  \
12  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                        structural_fp  min_Km_for_EC  \
12  1100111100000001001000110110010001001111111100...   3.000000e-07   

                                             MACCS FP frac_of_min_UID  \
12  0000000000000000000000000000000000000000000000...        0.676056   

   frac_of_min_RID  frac_of_min_EC  
12             1.0        0.042254  

[1 rows x 28 columns]
      ECs              Organism Uniprot IDs      PMID      Type        Km

In [31]:
df_new2 = pd.DataFrame(data = {"Reaction ID" : df_validation["Reaction ID"],
                                  "Sequence ID" : df_validation["Sequence ID"],
                                  "Temperature" : df_validation["Temperature"],
                                    "pH" : df_validation["pH"],
                                  "Type" : df_validation["Type"],
                                  "Main substrate" : df_validation["MACCS FP"]})

df_new2.drop_duplicates(inplace = True)
df_new2.reset_index(inplace = True, drop = True)

df_new2["Km_values"], df_new2["Uniprot IDs"], df_new2["ECs"], df_new2["Organisms"], df_new2["Substrates"], df_new2["Products"], df_new2["ESM2"], df_new2["Sequence"], df_new2["difference_fp"], df_new2["structural_fp"] = "", "", "", "", "", "", "", "", "", ""

for ind in df_new2.index:
    RID, SID, Temp, pH, Type, MSubstrate = df_new2["Reaction ID"][ind], df_new2["Sequence ID"][ind], df_new2["Temperature"][ind], df_new2["pH"][ind], df_new2["Type"][ind], df_new2["Main substrate"][ind]
    help_df = df_validation.loc[df_validation["Reaction ID"] 
                              == RID].loc[df_validation["Sequence ID"] 
                                          == SID].loc[df_validation["Temperature"] 
                                                      == Temp].loc[df_validation["pH"] 
                                                                    == pH].loc[df_validation["Type"] 
                                                                              == Type].loc[df_validation["MACCS FP"] 
                                                                                                            == MSubstrate]
    df_new2["ECs"][ind] = list(help_df["ECs"])
    df_new2["Km_values"][ind] = list(help_df["Km"])
    df_new2["Uniprot IDs"][ind] = list(help_df["Uniprot IDs"])
    df_new2["Organisms"][ind] = list(help_df["Organism"])
    df_new2["Type"][ind]
    df_new2["Sequence"][ind] = help_df["Sequence"].values[0]
    df_new2["ESM2"][ind] = help_df["Enzyme rep"].values[0]
    df_new2["difference_fp"][ind], df_new2["structural_fp"][ind] = help_df["difference_fp"].values[0], help_df["structural_fp"].values[0]
    df_new2["Substrates"][ind], df_new2["Products"][ind] = help_df["Substrates"].values[0], help_df["Products"].values[0]

In [32]:
df_arabidopsis = df_new
df_validation = df_new2

In [33]:
df_arabidopsis["geomean_Km"] = np.nan
for ind in df_arabidopsis.index:
    all_Km = np.array(df_arabidopsis["Km_values"][ind]).astype(float)
    min_Km = min(all_Km)
    all_Km_top = [Km for Km in all_Km  if min_Km/Km >= 0.01]
    df_arabidopsis["geomean_Km"][ind] = np.mean((all_Km_top))

C:\Users\marle\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
df_validation["geomean_Km"] = np.nan
for ind in df_validation.index:
    all_Km = np.array(df_validation["Km_values"][ind]).astype(float)
    min_Km = min(all_Km)
    all_Km_top = [Km for Km in all_Km  if min_Km/Km >= 0.01]
    df_validation["geomean_Km"][ind] = np.mean((all_Km_top))
    
df_validation.to_pickle(join("..", "..", "data", "validation_%s.pkl" %organism))

C:\Users\marle\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
df = df_Km.copy()
df = df.sample(frac = 1, random_state = 123)
df.reset_index(drop= True, inplace = True)

train_df, test_df = split_dataframe_enzyme(frac = 5, df = df.copy())
print("Test set size: %s" % len(test_df))
print("Training set size: %s" % len(train_df))
print("Size of test set in percent: %s" % np.round(100*len(test_df)/ (len(test_df) + len(train_df))))


train_df.reset_index(inplace = True, drop = True)
test_df.reset_index(inplace = True, drop = True)

train_df.to_pickle(join("..", "..", "data", "train_df_Km_%s.pkl" %organism))
test_df.to_pickle(join("..", "..", "data", "test_df_Km_%s.pkl" %organism))

Test set size: 105
Training set size: 517
Size of test set in percent: 17.0


In [25]:
data_train2 = train_df.copy()
data_train2["index"] = list(data_train2.index)

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=5)
indices_fold1 = list(df_fold["index"])
print(len(data_train2), len(indices_fold1))#

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=4)
indices_fold2 = list(df_fold["index"])
print(len(data_train2), len(indices_fold2))

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=3)
indices_fold3 = list(df_fold["index"])
print(len(data_train2), len(indices_fold3))

data_train2, df_fold = split_dataframe_enzyme(df = data_train2, frac=2)
indices_fold4 = list(df_fold["index"])
indices_fold5 = list(data_train2["index"])
print(len(data_train2), len(indices_fold4))


fold_indices = [indices_fold1, indices_fold2, indices_fold3, indices_fold4, indices_fold5]

train_indices = [[], [], [], [], []]
test_indices = [[], [], [], [], []]

for i in range(5):
    for j in range(5):
        if i != j:
            train_indices[i] = train_indices[i] + fold_indices[j]
    test_indices[i] = fold_indices[i]
    
    
np.save(join("..", "..", "data", "CV_train_indices_%s" %organism), train_indices)
np.save(join("..", "..", "data", "CV_test_indices_%s" %organism), test_indices)

429 88
336 93
226 110
111 115
